In [1]:

'''
    0. Initialise with cols list of lists (list of columns)
    COLUMN
    1. Create list for headers from Order field names
    2. Add to the list, with product header items
    3. create list for customer details
    4. On the same list index, append product counts for that customer
    5. Total row
    6. formatting
'''
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
# 0. Cols is to be the eventual list of lists
cols = []

In [3]:
# 1. Create list for headers from Order field names
import pandas as pd
from collections import OrderedDict
from order.models import Order

# Customer Headers Name, Address, Postcode, Email etc
headers = []
verbose_field_names = []
order_fields = ['customer_name','customer_address', 'customer_postcode', 'customer_email','customer_phone', 'fulfillment_method', 'collection_location','notes']

for h in order_fields:
    verbose_field_names.append(Order._meta.get_field(h).verbose_name)



In [40]:
# 2. Add to the list, with product header items
from product.models import Product

qs = Product.objects.filter(published=True)
product_names = list(qs.values_list('name', flat=True))
product_ids = list(qs.values_list('id', flat=True))

ids_names = list(qs.values('id','name'))

product_names = [ p.get('name') for p in ids_names]
product_ids = [ p.get('id') for p in ids_names]


In [41]:
# 3. create list for customer details
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
f_event_id = 2
qs = Order.objects.filter(fulfillment_event_id=f_event_id)
fields = qs.values_list(*order_fields)

cols = [verbose_field_names + product_names]
for obj, cust_order_fields in zip(qs,fields):
    customer_order_product_counts = tuple([obj.product_count(p_id) for p_id in product_ids])
    cols.append(cust_order_fields + customer_order_product_counts)
    

In [42]:
# Convert multidimensional array to dataframe via transposing

df = pd.DataFrame(cols)
output = df.transpose()


In [43]:
# Write xlsx to disk
import xlsxwriter


# Create a Pandas Excel writer using XlsxWriter as the engine.
# Saved to api/input.xlsx
writer = pd.ExcelWriter('input.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
output.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [8]:
# Format the xlsx by removing row 1 and col A

# Row 1 height for Address needs to be 5x the size

# non-header columns need to have border lines